In [6]:
import sys
# sys.path.append(r"..\backend\app\services")
sys.path.append(r"..apps\backend")
from services.ai_service import AIService
# from ..apps.backend.app import main

ModuleNotFoundError: No module named 'services'

In [2]:
pip install pandas

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 61.4 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

df = pd.read_csv("https://ptsbrkwalysbchtdharj.supabase.co/storage/v1/object/public/test-bucket/mr46r49qgzi_1745310666295.csv")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>

In [123]:
import pandas as pd
import base64
import io
import plotly.io as pio
import threading
import sqlite3

In [12]:
data = pd.read_csv("input.csv")
ai_service = AIService()

In [11]:
data

,month,amount,subsidiary_name,region
0,2024-01-01,125000,Alpha Corp,North America
1,2024-02-01,98000,Beta Ltd,Europe
2,2024-03-01,112000,Gamma Inc,Asia
3,2024-04-01,134000,Delta LLC,South America
4,2024-05-01,89000,Epsilon Co,Africa
...,...,...,...,...
67,2024-08-01,107000,Alpha Beta Gamma Delta,Europe
68,2024-09-01,126000,Beta Gamma Delta Epsilon,Asia
69,2024-10-01,151000,Gamma Delta Epsilon Zeta,South America
70,2024-11-01,95000,Delta Epsilon Zeta Theta,Africa


In [148]:
def validate_data(data, query=None, context=None, ai_service=None):
        validation_output = []
        # validation_output.append("Validating your data...")

        # Check for missing values
        if data.isnull().sum().sum() > 0:
            validation_output.append("The following columns have empty values:")
            validation_output.append(data.isnull().sum().to_dict())
        else:
            validation_output.append("No empty values found!")

        # Check for duplicates
        if data.duplicated().sum() > 0:
            dup_num = data.duplicated().sum()
            validation_output.append(f"{dup_num} duplicate entries found.")
        else:
            validation_output.append("No duplicate entries found.")

        # Validate data types
        # validation_output.append("Validating the data types...")
        data['month'] = pd.to_datetime(data['month'], errors='coerce')
        return validation_output

def get_insights(data, query, context, ai_service):
        insights_output = []
        # print("Going for insights")
        data_text = data.to_string(index=False)
        # context = f"The following is the financial data for 2024:\n{data_text}"
        final_query = f"Generate valuable business insights about the given data: {data_text}, trying to best answer the following  query: {query}. \
        Be specific and provide actionable insights. "
        response = ai_service.process_query(context, final_query)
        # print(response)
        insights_output.append(response)

        return insights_output

class TimeoutException(Exception):
    pass

def exec_with_timeout(code, local_vars, timeout=5):
    def target():
        try:
            exec(code, {}, local_vars)
        except Exception as e:
            local_vars["error"] = e

    thread = threading.Thread(target=target)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        raise TimeoutException("Execution timed out")
    if "error" in local_vars:
        raise local_vars["error"]

def visualize(data, query, context, ai_service):
        if not query:
            query = "Given the data, provide me a valuable visualization"
        visualization_output = []
        viz_components = {
             "barchart": ["x", "y", "color", "title", "xaxis_title", "yaxis_title"],
             "piechart": ["values", "names", "title"],
                "scatter": ["x", "y", "color", "title", "xaxis_title", "yaxis_title"],
                "line": ["x", "y", "title", "xaxis_title", "yaxis_title"],
                "heatmap": ["z", "x", "y", "title", "xaxis_title", "yaxis_title"],
        }

        init_queries = [
             f"I want you to decide what's the best visualization type to use in order to answer the following question: {query} \
             Could be a barchart, piechart, scatter, line, or heatmap. \
                Print only the name of the visualization type and nothing else. ",

                f"Given the following data: {data.head(10).to_string(index=False)} I want you to give me the following components for a nice visualization \
                    Print only the components and nothing else in the following format: \
                    <component1>: <value1>, <component2>: <value2>, <component3>: <value3>, ...\n " 
    
        ]
        
        data_text = data.to_string(index=False)
        # context = f"The following is the financial data for 2024:\n{data_text}"

        # First LLM Pass to get the visualization type
        query_type = ai_service.process_query(context, init_queries[0])

        if query_type not in viz_components.keys(): 
             query_type = "barchart"

        # Second LLM Pass to get the visualization components
        query_components = ai_service.process_query(context, init_queries[1] + "I am looking to creating a " + query_type + ". The components I need: " + str(viz_components[query_type]))
        # Transform the query components into a dictionary
        query_components = query_components.split(",") 
        query_components = [x.strip() for x in query_components]
        query_components = {k: v.split(": ")[1] for k, v in zip(viz_components[query_type], query_components)}

        visualization_output.append(query_components)
        
        return visualization_output

def calculate(data, query, context, ai_service):
        calculation_output = []
        data_columns = data.columns
        final_query = f"Given a table with the following metadata: {data_columns} \
              I want you to generate a SQL code, trying to answer the following question: {query}. \
              Assume that the table name is 'data_table'. \
              Provide the SQL code only, and nothing else."
        response = ai_service.process_query(context, final_query)

        # Create an in-memory SQLite database
        conn = sqlite3.connect(":memory:")
        print(f"Calculate response: {response}")
        try:
            # Load the DataFrame into the SQLite database
            data.to_sql("data_table", conn, index=False, if_exists="replace")

            # Execute the SQL query
            result = pd.read_sql_query(response, conn)

            return result
        except Exception as e:
            print(f"Error executing SQL query: {e}")
            return None
        finally:
            # Close the connection
            conn.close()

        calculation_output.append(result)

        return calculation_output

In [149]:
tools_funcs = {
    "validate_data": validate_data,
    "get_insights": get_insights,
    "visualize": visualize,
    "calculate": calculate,
}

tools_explanation = {
    "validate_data": "Check for missing values, duplicates, and data types.",
    "get_insights": "Generate valuable business insights about the given data.",
    "visualize": "Generate a visualization of the data.",
    "calculate": "Performs calculations based on a user question."
}

context = f"You are an experienced data analyst, expert in giving quality information and insights about various data types. \
    I will be giving you a dataset, and you will be providing quality deiliverables. You have the following tools available: {{tools}}. " \

master_query = f"Given the following dataset: {{data}}, a user query {{user_query}}, I want you to decide which tool to use in order to answer the user query. " \
"Print only the name of the tool and nothing else. " 

feedback_query = f"Given the user query {{user_query}}, you have used the following tools, producing the respective responses: \
    {{tool_responses}} \
    Do you have enough information to answer the user query, or are additional information missing?  \
    If yes, print 'yes' and nothing else. \
    If not, print only the name of the tool you need to use in order to answer the user query, and nothing else. Don't price NO in this case. \
    Available tools are {{tools}}. " 

response_query = f"Given the user query {{user_query}}, you have used the following tools, producing the respective responses: \
    {{tool_responses}} \
    Using the above, please provide the user with a proper response. \
    Be as precise as possible, and only answer exactlt what is asked in the user query. \
    Don't mention any information about the tools you have used."  

In [156]:
data

,Year,Champion,Regular Season Wins
0,1980,Los Angeles Lakers,60
1,1981,Boston Celtics,62
2,1982,Los Angeles Lakers,57
3,1983,Philadelphia 76ers,65
4,1984,Boston Celtics,62
5,1985,Los Angeles Lakers,62
6,1986,Boston Celtics,67
7,1987,Los Angeles Lakers,65
8,1988,Los Angeles Lakers,62
9,1989,Detroit Pistons,63


In [155]:
calculate(data, user_query, context_upd, ai_service)

KeyError: 0

In [150]:
user_query = "What is the monthly revenue per region?"
user_query = "What is the total revenue per month?"
user_query = "Give me the 5 top teams with the highest average number of which when they won the champion."
data = pd.read_csv("nba_champions_1980_onwards.csv")
# data = pd.read_csv("input.csv")

In [151]:
# response = "validate_data"

# tools[response](data, ai_service)

In [152]:
first_time_flag = True
cur_response = None
counter = 0

tools_responses = {}

data_text = data.to_string(index=False)
context_upd = context.format(tools=str(list(tools.keys())))

tools_expl = ", ".join([f"{k}: {v}" for k, v in tools_explanation.items()])

while True: 
    counter += 1
    if first_time_flag:
        first_time_flag = False
        cur_tool = ai_service.process_query(context_upd, master_query.format(data=data_text, user_query=user_query)).replace("'", "")
    else:
        tool_responses = ", ".join([f"Iteration {k}: Tool: {v[0]}, Response: {v[1]}" for k, v in tools_responses.items()])
        # print(feedback_query.format(user_query=user_query, tool_responses=tool_responses))
        cur_tool = ai_service.process_query(context_upd, feedback_query.format(user_query=user_query, tool_responses=tool_responses, tools=tools_expl)).replace("'", "")
        
        if cur_tool == "yes":
            print("The tool has given you a proper response.")
            final_response = ai_service.process_query(context_upd, response_query.format(user_query=user_query, tool_responses=tool_responses))
            print(final_response)
            break
        else:
            cur_tool = cur_tool.strip()
    print(f"Iteration {counter}")
    print(f"Using tool: {cur_tool}")
    cur_response = tools_funcs[cur_tool](data, user_query, context_upd, ai_service)
    tools_responses[str(counter)] = [cur_tool, cur_response]
    # print(f"Response: {cur_response}")


Iteration 1
Using tool: visualize
Iteration 2
Using tool: calculate
Calculate response: SELECT Champion, AVG(RS_Wins) AS Average_RS_Wins
FROM (
    SELECT Champion, SUM(RS_Wins) AS RS_Wins
    FROM data_table
    GROUP BY Champion
) AS subquery
ORDER BY Average_RS_Wins DESC
LIMIT 5;
Error executing SQL query: Execution failed on sql 'SELECT Champion, AVG(RS_Wins) AS Average_RS_Wins
FROM (
    SELECT Champion, SUM(RS_Wins) AS RS_Wins
    FROM data_table
    GROUP BY Champion
) AS subquery
ORDER BY Average_RS_Wins DESC
LIMIT 5;': no such column: RS_Wins
Iteration 3
Using tool: calculate
Calculate response: SELECT Champion, AVG(RS_Wins) AS Average_RS_Wins
FROM (
    SELECT Champion, SUM(RS_Wins) AS RS_Wins
    FROM data_table
    GROUP BY Champion
) AS subquery
ORDER BY Average_RS_Wins DESC
LIMIT 5;
Error executing SQL query: Execution failed on sql 'SELECT Champion, AVG(RS_Wins) AS Average_RS_Wins
FROM (
    SELECT Champion, SUM(RS_Wins) AS RS_Wins
    FROM data_table
    GROUP BY Champi

KeyError: 0

In [112]:
tool_responses

"Iteration 1: Tool: visualize, Response: [{'x': 'month', 'y': 'amount', 'color': 'subsidiary_name', 'title': 'Regional Sales by Subsidiary', 'xaxis_title': 'Month', 'yaxis_title': 'Amount'}], Iteration 2: Tool: get_insights, Response: ['To generate valuable business insights about the given data, I will use the following tools: `validate_data`, `get_insights`, and `visualize`.\\n\\n**Data Validation:**\\nThe provided data appears to be a clean and well-structured dataset with no missing values. The `validate_data` tool confirms that the data is valid and ready for analysis.\\n\\n**Insights:**\\nTo answer the query, I will first calculate the monthly revenue per region. Here are the results:\\n\\n| Region | Monthly Revenue |\\n| --- | --- |\\n| North America | $124,500 |\\n| Europe | $97,500 |\\n| Asia | $112,500 |\\n| South America | $137,500 |\\n| Africa | $91,500 |\\n| Australia | $151,500 |\\n\\nFrom these results, we can draw the following insights:\\n\\n1. **Regional Revenue Distr

In [96]:
ai_service.process_query(context_upd, feedback_query.format(user_query=user_query, tool_responses=tool_responses, tools=str(list(tools.keys())))).replace("'", "")

'no\nget_insights'

In [69]:
ai_service.process_query(context_upd, master_query.format(data=data_text, user_query=user_query)).replace("'", "")

'validate_data'

In [80]:
", ".join([f"Iteration {k}: Tool {v[0]}, Response: {v[1]}" for k, v in tools_responses.items()])

"Iteration 1: Tool validate_data, Response: ['No empty values found!', 'No duplicate entries found.']"

In [62]:
context

'You are an experienced data analyst, expert in giving quality information and insights about various data types. I will be giving you a dataset, and you will be providing quality deiliverables. You have the following tools available: {{tools}}. '

In [53]:
master_query.format(tools=tools, data=data_text, user_query=user_query)

'Given the following dataset:      month  amount          subsidiary_name        region\n2024-01-01  125000               Alpha Corp North America\n2024-02-01   98000                 Beta Ltd        Europe\n2024-03-01  112000                Gamma Inc          Asia\n2024-04-01  134000                Delta LLC South America\n2024-05-01   89000               Epsilon Co        Africa\n2024-06-01  145000               Zeta Group     Australia\n2024-07-01  123000        Theta Enterprises North America\n2024-08-01   97000            Iota Holdings        Europe\n2024-09-01  110000               Kappa Corp          Asia\n2024-10-01  140000               Lambda Ltd South America\n2024-11-01   87000                   Mu Inc        Africa\n2024-12-01  150000                   Nu LLC     Australia\n2024-01-01  128000               Omicron Co North America\n2024-02-01   95000                 Pi Group        Europe\n2024-03-01  115000          Rho Enterprises          Asia\n2024-04-01  137000        

In [56]:
master_query.format(tools=str(list(tools.keys())), data=data_text, user_query=user_query)

'Given the following dataset:      month  amount          subsidiary_name        region\n2024-01-01  125000               Alpha Corp North America\n2024-02-01   98000                 Beta Ltd        Europe\n2024-03-01  112000                Gamma Inc          Asia\n2024-04-01  134000                Delta LLC South America\n2024-05-01   89000               Epsilon Co        Africa\n2024-06-01  145000               Zeta Group     Australia\n2024-07-01  123000        Theta Enterprises North America\n2024-08-01   97000            Iota Holdings        Europe\n2024-09-01  110000               Kappa Corp          Asia\n2024-10-01  140000               Lambda Ltd South America\n2024-11-01   87000                   Mu Inc        Africa\n2024-12-01  150000                   Nu LLC     Australia\n2024-01-01  128000               Omicron Co North America\n2024-02-01   95000                 Pi Group        Europe\n2024-03-01  115000          Rho Enterprises          Asia\n2024-04-01  137000        

In [142]:
import pandas as pd

# Data: NBA Champions and their regular season wins from 1980 onwards
data = {
    "Year": [
        1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
        1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
        2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
        2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
        2020, 2021, 2022
    ],
    "Champion": [
        "Los Angeles Lakers", "Boston Celtics", "Los Angeles Lakers", "Philadelphia 76ers", "Boston Celtics",
        "Los Angeles Lakers", "Boston Celtics", "Los Angeles Lakers", "Los Angeles Lakers", "Detroit Pistons",
        "Detroit Pistons", "Chicago Bulls", "Chicago Bulls", "Chicago Bulls", "Houston Rockets", "Houston Rockets",
        "Chicago Bulls", "Chicago Bulls", "Chicago Bulls", "San Antonio Spurs", "Los Angeles Lakers",
        "Los Angeles Lakers", "Los Angeles Lakers", "San Antonio Spurs", "Detroit Pistons", "San Antonio Spurs",
        "Miami Heat", "San Antonio Spurs", "Boston Celtics", "Los Angeles Lakers", "Los Angeles Lakers",
        "Dallas Mavericks", "Miami Heat", "Miami Heat", "San Antonio Spurs", "Golden State Warriors",
        "Cleveland Cavaliers", "Golden State Warriors", "Golden State Warriors", "Toronto Raptors",
        "Los Angeles Lakers", "Milwaukee Bucks", "Golden State Warriors"
    ],
    "Regular Season Wins": [
        60, 62, 57, 65, 62, 62, 67, 65, 62, 63, 59, 61, 67, 57, 58, 47, 72, 69, 62, 37,
        67, 56, 58, 60, 54, 59, 52, 58, 66, 65, 57, 57, 66, 54, 67, 73, 67, 58, 53, 52,
        56, 51, 53
    ]
}

# Create a DataFrame
nba_champions = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = "nba_champions_1980_onwards.csv"
nba_champions.to_csv(csv_file_path, index=False)